In [34]:
import numpy as np
import mdtraj as md
import nglview as nv

# Data

In [11]:

pdb_path = f"/home/shpark/prj-mlcv/lib/DESRES/data/CLN025.pdb"
traj_pdb = md.load_pdb(pdb_path)

view_pdb = nv.show_mdtraj(traj_pdb)
view_pdb

non_h_atoms = [atom for atom in traj_pdb.topology.atoms if atom.element.symbol != 'H']
num_non_h = len(non_h_atoms)
print(f"Number of non-hydrogen atoms: {num_non_h}")
print(non_h_atoms)

view_pdb.clear_representations()
# view_pdb.add_representation("ball+stick", selection="not hydrogen")
# view_pdb.add_representation("licorice", selection="not hydrogen")
view_pdb.add_representation("licorice", selection="protein")
view_pdb

Number of non-hydrogen atoms: 93
[TYR1-N, TYR1-CA, TYR1-CB, TYR1-CG, TYR1-CD1, TYR1-CE1, TYR1-CZ, TYR1-OH, TYR1-CD2, TYR1-CE2, TYR1-C, TYR1-O, TYR2-N, TYR2-CA, TYR2-CB, TYR2-CG, TYR2-CD1, TYR2-CE1, TYR2-CZ, TYR2-OH, TYR2-CD2, TYR2-CE2, TYR2-C, TYR2-O, ASP3-N, ASP3-CA, ASP3-CB, ASP3-CG, ASP3-OD1, ASP3-OD2, ASP3-C, ASP3-O, PRO4-N, PRO4-CD, PRO4-CA, PRO4-CB, PRO4-CG, PRO4-C, PRO4-O, GLU5-N, GLU5-CA, GLU5-CB, GLU5-CG, GLU5-CD, GLU5-OE1, GLU5-OE2, GLU5-C, GLU5-O, THR6-N, THR6-CA, THR6-CB, THR6-OG1, THR6-CG2, THR6-C, THR6-O, GLY7-N, GLY7-CA, GLY7-C, GLY7-O, THR8-N, THR8-CA, THR8-CB, THR8-OG1, THR8-CG2, THR8-C, THR8-O, TRP9-N, TRP9-CA, TRP9-CB, TRP9-CG, TRP9-CD1, TRP9-NE1, TRP9-CE2, TRP9-CD2, TRP9-CE3, TRP9-CZ3, TRP9-CZ2, TRP9-CH2, TRP9-C, TRP9-O, TYR10-N, TYR10-CA, TYR10-CB, TYR10-CG, TYR10-CD1, TYR10-CE1, TYR10-CZ, TYR10-OH, TYR10-CD2, TYR10-CE2, TYR10-C, TYR10-O, TYR10-OXT]


NGLWidget()

# Samples

In [ ]:
# method = "org-0707-0536"
method = "ours-0701_1147"

xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/samples.xtc"
pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/topology.pdb"
reconstructed_xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/samples_sidechain_rec.xtc"
reconstructed_pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/samples_sidechain_rec.pdb"

traj = md.load(reconstructed_xtc_path, top=reconstructed_pdb_path)
print(traj)

view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
# view.add_representation("licorice", selection="not hydrogen")
view

<mdtraj.Trajectory with 31 frames, 92 atoms, 10 residues, without unitcells>


NGLWidget(max_frame=30)

In [58]:
xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/samples_md_equil.xtc"
pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025-{method}/samples_md_equil.pdb"

traj = md.load(xtc_path, top=pdb_path)
print(traj)

view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view

<mdtraj.Trajectory with 31 frames, 93 atoms, 10 residues, without unitcells>


NGLWidget(max_frame=30)

In [59]:
view.frame = 0

In [60]:
def foldedness_by_hbond(
    traj,
    distance_cutoff=0.35,
    bond_number_cutoff=3
):
	"""
	Generate binary labels for folded/unfolded states based at least 3 bonds among eight bonds
	- TYR1T-YR10OT1
	- TYR1T-YR10OT2
	- ASP3N-TYR8O
	- THR6OG1-ASP3O
	- THR6N-ASP3OD1
	- THR6N-ASP3OD2
	- TYR10N-TYR1O


	Args:
		traj (mdtraj): mdtraj trajectory object
		distance_cutoff (float): donor-acceptor distance cutoff in nm (default 0.35 nm = 3.5 amstrong)
		angle_cutoff (float): hydrogen bond angle cutoff in degrees (default 110 deg)
		bond_number_cutoff (int): minimum number of bonds to be considered as folded (default 3)

	Returns:
		labels (np.array): binary array (1: folded, 0: unfolded)
	"""
	# TYR1N-YR10OT1
	donor_idx = traj.topology.select('residue 1 and name N')[0] # Tyr1:N
	acceptor_idx = traj.topology.select('residue 10 and name O')[0]   # Tyr10:OT1
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_O1 = ((distance[:,0] < distance_cutoff)).astype(int)
	label_O2 = ((distance[:,0] < distance_cutoff)).astype(int) 
	label_O3 = ((distance[:,0] < distance_cutoff)).astype(int)
	label_TYR1N_TYR10OT1 = label_O1 | label_O2 | label_O3


	# TYR1N-YR10OT2
	donor_idx = traj.topology.select('residue 1 and name N')[0] # Tyr1:N
	acceptor_idx = traj.topology.select('residue 10 and name OXT')[0]   # Tyr10:OT2
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_O1 = ((distance[:,0] < distance_cutoff)).astype(int)
	label_O2 = ((distance[:,0] < distance_cutoff)).astype(int)
	label_O3 = ((distance[:,0] < distance_cutoff)).astype(int)
	label_TYR1N_TYR10OT2 = label_O1 | label_O2 | label_O3


	# ASP3N-TYR8O
	donor_idx = traj.topology.select('residue 3 and name N')[0]
	acceptor_idx = traj.topology.select('residue 8 and name O')[0]
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_ASP3N_TYR8O = ((distance[:,0] < distance_cutoff)).astype(int)
 
 
	# THR6OG1-ASP3O
	donor_idx = traj.topology.select('residue 6 and name OG1')[0]
	acceptor_idx = traj.topology.select('residue 3 and name O')[0]
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_THR6OG1_ASP3O = ((distance[:,0] < distance_cutoff)).astype(int)
 
 
	# THR6N-ASP3OD1
	donor_idx = traj.topology.select('residue 6 and name N')[0]
	acceptor_idx = traj.topology.select('residue 3 and name OD1')[0]
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_THR6N_ASP3OD1 = ((distance[:,0] < distance_cutoff)).astype(int)
 
	# THR6N-ASP3OD2
	donor_idx = traj.topology.select('residue 6 and name N')[0]
	acceptor_idx = traj.topology.select('residue 3 and name OD2')[0]
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_THR6N_ASP3OD2 = ((distance[:,0] < distance_cutoff)).astype(int)
 
 
	# GLY7N-ASP3O
	donor_idx = traj.topology.select('residue 7 and name N')[0]
	acceptor_idx = traj.topology.select('residue 3 and name O')[0]
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_GLY7N_ASP3O = ((distance[:,0] < distance_cutoff)).astype(int)
 

	# TYR10N-TYR1O
	donor_idx = traj.topology.select('residue 10 and name N')[0] 
	acceptor_idx = traj.topology.select('residue 1 and name O')[0] 
	distance = md.compute_distances(traj, [[donor_idx, acceptor_idx]])
	label_TYR10N_TYR1O = ((distance[:,0] < distance_cutoff)).astype(int)




	# ASP3OD_THR6OG1_ASP3N_THR8O
	bond_sum = label_TYR1N_TYR10OT1 + label_TYR1N_TYR10OT2 + label_ASP3N_TYR8O + label_THR6OG1_ASP3O \
		+ label_THR6N_ASP3OD1 + label_THR6N_ASP3OD2 + label_GLY7N_ASP3O + label_TYR10N_TYR1O
	labels = bond_sum >= bond_number_cutoff

	# print(label_TYR10OT_TYR1N.sum())
	# print(labels_TYR10N_TYR1O.sum())
	# print(labels_ASP3OD_THR6OG1.sum())
	# print(labels_ASP3N_TYR8O.sum())

	return labels, bond_sum

In [61]:
label, bond_sum = foldedness_by_hbond(traj)

print(f"{label.sum()} folded states out of {label.shape} total states")
print(f"Folded states: {np.where(label)[0]}")

21 folded states out of (31,) total states
Folded states: [ 0  1  2  4  5  6  7  8 13 14 15 16 19 20 22 23 24 25 26 28 30]
